In [1]:
import camelot
import pandas as pd
import numpy as np
import tabula
import re

In [2]:
x=camelot.read_pdf("BTECH21-Grade-sheet.pdf",pages='all',flavor='stream',edge_tol=1,columns=['2']*8)

In [3]:
len(x)

143

In [88]:
def row(roll_index,index,x):
    roll=re.search("\d\dVV\dA\d\d\d\d",x.df.iloc[roll_index,1].splitlines()[0]).group()
    l=x.df.iloc[index,1].splitlines()
    l.append(roll)
    l[4]=l[4][0]
    y=np.reshape(l,(1,6))
    return [y,roll_index,index+1]
#l,b,c=row(roll_index,index)

In [89]:
df=pd.DataFrame()
def f(x):
    roll_index=3
    index=roll_index+2
    df=pd.DataFrame(columns=['Sno', 'SUBCODE', 'Subname', 'Grade', 'Credits',"Htno"])
    while(index<=len(x.df.iloc[:,1])):
        z=x.df.iloc[index,1].splitlines()[0]
        y="TOTAL"
        i=0
        while(z!=y):
            #print(1)
            l,roll_index,index=row(roll_index,index,x)
            a=pd.DataFrame(l,columns=['Sno', 'SUBCODE', 'Subname', 'Grade', 'Credits',"Htno"])
            df=df.append(a)
            #print(df.iloc[:,:])
            i=i+1
            z=x.df.iloc[index,1].splitlines()[0]
        roll_index=index+4
        index=roll_index+2
        #print(df)
    return pd.DataFrame(df)
for i in range(len(x)):
    df=df.append(f(x[i]))

In [90]:
#df.rename(columns={'SUBNAME':'Subname','GRADE':'Grade','CREDITS':'Credits','roll_no':'Htno','NO':'Sno'},inplace=True)
df.drop('SUBCODE',axis=1,inplace=True)
df.set_index('Sno',inplace=True)

In [91]:
df['Credits']=df['Credits'].astype('int')

In [92]:
df.head()

,Subname,Grade,Credits,Htno
Sno,,,,
1,PROBABILITY & STATISTICS,F,0,16VV1A1203
2,MATHEMATICAL FOUNDATIONS OF COMPUTER SCIENCE,F,0,16VV1A1203
3,DATA STRUCTURES THOROUGH C++,F,0,16VV1A1203
4,DIGITAL LOGIC DESIGN,F,0,16VV1A1203
5,PYTHON PROGRAMMING,D,3,16VV1A1203


In [93]:
#chainging to requried format
df['points']=df['Grade'].map({'O':10,'S':9,'A':8,'B':7,'C':6,'D':5,'F':0,"ABSENT":0})
#df['Credits']=df['Credits'].apply(lambda x:int(str(x).split(" ")[0]))
df['Total']=df['points']*df['Credits']
#df.drop(['Grade','points'],axis=1,inplace=True)
df['sep']=df['Htno'].apply(lambda x:x[6:8])
df['year']=df['Htno'].apply(lambda x:x[0:2])
df['sep']=df['sep'].astype('int')
df['year']=df['year'].astype('int')

In [94]:
present_year=int(input())

18


In [95]:
df_s=df[df['year']<present_year]
df_r=df[df['year']>=present_year]

In [96]:
#division based on roll numbers
def seperate_branch(df,seperate):
    unique=df['sep'].unique()
    for i in range(len(unique)):
        seperate.append(df[df.sep==unique[i]])

In [97]:
seperate_r=[]
seperate_s=[]
seperate_branch(df_r,seperate_r)
seperate_branch(df_s,seperate_s)

In [98]:
#requried format
def req_table(seperate,table):
    for i in range(len(seperate)):
        table.append(seperate[i].pivot_table(index='Htno',columns='Subname',values='points',fill_value=0))
        table[i]['Credits']=seperate[i].groupby('Htno')['Credits'].sum()
        table[i]["SGPA"]=seperate[i].groupby('Htno')['Total'].sum()/table[i]['Credits']

In [99]:
table_r=[]
table_s=[]
req_table(seperate_r,table_r)
req_table(seperate_s,table_s)

In [100]:
def grad(table):
    grade={10:'O',9:'S',8:'A',7:'B',6:'C',5:'D',0:'F'}
    for i in range(len(table)):
        col=list(table[i].columns).index('Credits')
        table[i].iloc[:,:col]=table[i].iloc[:,:col].replace(grade)
        table[i].reset_index(inplace=True)
        table[i].index = np.arange(1, len(table[i]) + 1)
        table[i]=table[i].rename_axis('SNo',axis=1)

In [101]:
grad(table_r)
grad(table_s)

In [104]:
def analysis(an,pp,table):
    for i in range(len(table)):
        n=list(table[i].columns).index('Htno')
        m=list(table[i].columns).index('Credits')
        analysis={
        'O grade':list((table[i]=='O').sum())[1:-2],
        'S grade':list((table[i]=='S').sum())[1:-2],
        'A grade':list((table[i]=='A').sum())[1:-2],
        'B grade':list((table[i]=='B').sum())[1:-2],
        'C grade':list((table[i]=='C').sum())[1:-2],
        'D grade':list((table[i]=='D').sum())[1:-2],
        'Fail':list((table[i]=='F').sum())[1:-2]
        }
        c=pd.DataFrame(analysis)
        c=c.T
        #print(n,m)
        c.columns=(table[i].columns)[n+1:m]
        an.append(c)
        pa={'pass':[len(table[i][table[i]['Credits']==22]['Credits'])],
         'pass per':100-int((len(table[i][table[i]['Credits']<22]['Credits'])/len(table[i]))*100)
        }
        d=pd.DataFrame(pa)
        d=d.T
        pp.append(d)

In [105]:
an_r=[]
an_s=[]
pp_r=[]
pp_s=[]
analysis(an_r,pp_r,table_r)
analysis(an_s,pp_s,table_s)

c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform eleme

In [108]:
#colour cells with 0 values 'fail'
def highlight(styler_obj,table):
    for i in range(len(table)):
        styler_obj.append(table[i].style.applymap(lambda x: 'background-color : #FFFF99' if x=='F' else ''))

In [109]:
styler_r=[]
styler_s=[]
highlight(styler_r,table_r)
highlight(styler_s,table_s)

In [111]:
styler_r[4]

SNo,Htno,COMPUTER GRAPHICS,DATA STRUCTURES THOROUGH C++,DATA STRUCTURES THROUGH C++ LABORATORY,DIGITAL LOGIC DESIGN,MATHEMATICAL FOUNDATIONS OF COMPUTER SCIENCE,PROBABILITY & STATISTICS,PYTHON PROGRAMMING,PYTHON PROGRAMMING LABORATORY,Credits,SGPA
1,18VV1A0501,F,C,F,F,C,D,F,S,11,6.272727
2,18VV1A0502,C,F,C,C,C,C,C,O,19,6.421053
3,18VV1A0503,B,B,S,A,A,B,B,S,22,7.636364
4,18VV1A0504,S,B,O,S,S,S,A,O,22,8.772727
5,18VV1A0505,A,C,A,B,A,B,B,O,22,7.500000
6,18VV1A0506,S,B,A,S,S,B,A,O,22,8.318182
7,18VV1A0507,S,A,O,S,O,B,S,O,22,8.909091
8,18VV1A0508,S,A,A,S,S,B,S,O,22,8.590909
9,18VV1A0509,S,C,A,A,S,A,B,O,22,8.045455
10,18VV1A0510,S,D,F,F,B,F,B,S,14,7.285714


In [112]:
#writing to excel
def write_to_excel(table,styler_obj):
    for i in range(len(table)):
        writer = pd.ExcelWriter('grades'+str(i)+'.xlsx', engine='xlsxwriter')
        styler_obj[i].to_excel(writer,sheet_name='Sheet'+str(i))
        an[i].to_excel(writer,sheet_name='Sheet'+str(i),startrow=len(table[i])+2,index=True)
        pp[i].to_excel(writer,sheet_name='Sheet'+str(i), startrow=len(table[i])+len(an[i])+5,index=True,header=None)
    
        for j in range(1,len(an[i].columns)+1):
            dt=table[i].iloc[:,[0,j]]
            dt.to_excel(writer,sheet_name='Sheet'+str(i+j))
            c=pd.DataFrame(an[i].iloc[:,j-1])
            c.to_excel(writer,sheet_name='Sheet'+str(i+j),startrow=len(dt)+1,index=True)
            workbook=writer.book
            worksheet = writer.sheets['Sheet'+str(i+j)]
            chart= workbook.add_chart({'type': 'pie'})
            chart.set_title({'name':(an[i].columns)[j-1]})
            chart.add_series({
            'categories':['Sheet'+str(i+j), len(table[i])+2, 0,len(table[i])+len(an[i])+1, 0],
            'values': ['Sheet'+str(i+j), len(table[i])+2, 1, len(table[i])+len(an[i])+1, 1],
            })
            worksheet.insert_chart('D10', chart)
        writer.save()

In [113]:
write_to_excel(table_r,styler_r)
write_to_excel(table_s,styler_s)

IndexError: positional indexers are out-of-bounds